<a href="https://colab.research.google.com/github/gtbnhyujmj/Auto_Shorts_Maker/blob/main/%5BCopiloted%5D_Auto_Paste_PNG2MP4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 匯入必要的套件
import cv2  # OpenCV：處理影片讀寫
from PIL import Image  # Pillow：處理 PNG 圖片，支援透明度
import numpy as np  # 處理陣列資料，影像格式轉換用

In [2]:
def blend_with_alpha_fast(base_img, overlay_imgs):

    """
    將多個 overlay PNG 疊在 base_img 上，且利用 numpy 直接處理透明通道以加速運算
    base_img: OpenCV BGR 格式的單張影片幀
    overlay_imgs: 已經 resize 好的 PIL.Image 物件列表
    回傳：BGR 格式 numpy array，已完成合成
    """

    # 將 base_img 轉成 RGBA（支援透明度）
    base_rgba = cv2.cvtColor(base_img, cv2.COLOR_BGR2RGBA)  # 直接轉成 RGBA 格式

    for overlay in overlay_imgs:
        # 確認 overlay 也是 RGBA 格式
        overlay_rgba = np.array(overlay.convert('RGBA'))
        # 計算 alpha 遮罩：overlay alpha / 255.0
        alpha_overlay = overlay_rgba[..., 3:] / 255.0  # 取得 overlay 的 alpha 通道，並正規化
        alpha_base = base_rgba[..., 3:] / 255.0  # 取得 base 的 alpha 通道，並正規化
        # 新的 alpha 通道：overlay 疊加在原本的 alpha 上
        out_alpha = alpha_overlay + alpha_base * (1 - alpha_overlay)
        # RGB 通道做加權混合
        base_rgba[..., :3] = (overlay_rgba[..., :3] * alpha_overlay + base_rgba[..., :3] * alpha_base * (1 - alpha_overlay)) / (out_alpha + 1e-6)
        # 更新 alpha 通道
        base_rgba[..., 3:] = out_alpha * 255

    # 轉回 BGR（OpenCV 標準格式），丟棄 alpha 通道
    result_bgr = cv2.cvtColor(base_rgba, cv2.COLOR_RGBA2BGR)
    return result_bgr

In [3]:
def overlay_multiple_pngs_on_video_fast(video_path, overlay_paths, output_path='output.mp4'):

    """
    將多張 PNG 疊在 MP4 影片每一幀上，提升速度（numpy 批次運算，減少 PIL 轉換次數）
    video_path: 輸入影片路徑
    overlay_paths: PNG 圖片路徑列表
    output_path: 輸出影片路徑
    """

    cap = cv2.VideoCapture(video_path)  # 開啟影片檔案

    fps = int(cap.get(cv2.CAP_PROP_FPS))  # 取得影片每秒幀數
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # 取得影片寬度
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # 取得影片高度

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # 指定影片格式
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))  # 建立影片寫出物件

    # 讀入所有 PNG，resize 成和影片一樣大，加速後續批次處理
    overlay_images = [Image.open(path).resize((width, height), Image.LANCZOS) for path in overlay_paths]  # LANCZOS 品質較佳

    frame_idx = 0  # 幀數計數器
    while True:
        ret, frame = cap.read()  # 讀取一幀
        if not ret:  # 沒有更多幀時跳出
            break

        # 疊加所有 overlay PNG（批次 numpy 運算法）
        result_frame = blend_with_alpha_fast(frame, overlay_images)  # 直接傳入 PIL.Image 物件列表
        out.write(result_frame)  # 寫入輸出影片
        frame_idx += 1  # 幀數加一

    cap.release()  # 釋放影片讀取資源
    out.release()  # 釋放影片寫入資源
    print(f"影片儲存至：{output_path}")  # 完成訊息

In [4]:
# ====== 執行參數區 ======

In [8]:
if __name__ == "__main__":
    # 設定輸入影片路徑
    video_path = '/content/final_output5.mp4'

    # 設定 PNG 疊圖的路徑
    overlay_paths = [
        '/content/drive/MyDrive/shorts/characters/char3.png',
        '/content/drive/MyDrive/shorts/PNG_Overlayers/OverLayers1.png',
        # 你可以加入更多 PNG 圖片路徑
    ]

    # 執行主函式，將 PNG 疊在影片每一幀上
    overlay_multiple_pngs_on_video_fast(video_path, overlay_paths, output_path='final_output.mp4')

影片儲存至：final_output.mp4


In [7]:
# STU